### PyTorch Model for Sentence level Multimodal Sentiment Analysis 

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YOGA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\YOGA\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YOGA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import numpy as np
import pandas as pd
import nltk
import string
import os
import glob
from os import listdir
import re
from PIL import ImageFile
from skimage.io import imread, imshow
import sklearn
from nltk.stem import PorterStemmer
ps=PorterStemmer()
from nltk import FreqDist
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score, classification_report 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

### Captions processing 

In [2]:
d_frame = pd.read_csv(r"C:\Users\YOGA\Desktop\sentiment.csv")
d_frame['raw'] = d_frame['raw'].str.lower()
d_frame

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including french fries.
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,french fries are not a healthy food but it is ...
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"the plate has one of my favorite foods on it, ..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"it was disgusting food, not just bad food."
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,a plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,a dirty bathroom that has a dirty window made ...
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,a dirty bathroom that has a window in it.
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,a dirty bathroom that has a dirty window made ...


In [3]:
d_frame['tokenized'] = d_frame['raw'].apply(word_tokenize)
print(d_frame['tokenized'])


0        [a, plate, of, delicious, food, including, fre...
1        [french, fries, are, not, a, healthy, food, bu...
2        [the, plate, has, one, of, my, favorite, foods...
3        [it, was, disgusting, food, ,, not, just, bad,...
4        [a, plate, of, disgusting, food, found, at, a,...
                               ...                        
39194    [a, dirty, bathroom, that, has, a, dirty, wind...
39195    [a, dirty, bathroom, that, has, a, window, in,...
39196    [a, towel, that, is, on, a, rack, in, a, dirty...
39197    [a, dirty, bathroom, that, has, a, dirty, wind...
39198    [a, dirty, bathroom, that, has, a, window, in,...
Name: tokenized, Length: 39199, dtype: object


In [4]:
d_frame['raw'].isna().sum()

0

In [5]:
raw_list = d_frame['tokenized'].tolist()
raw_list

[['a',
  'plate',
  'of',
  'delicious',
  'food',
  'including',
  'french',
  'fries',
  '.'],
 ['french',
  'fries',
  'are',
  'not',
  'a',
  'healthy',
  'food',
  'but',
  'it',
  'is',
  'an',
  'excellent',
  'food',
  'for',
  'teenagers',
  '.'],
 ['the',
  'plate',
  'has',
  'one',
  'of',
  'my',
  'favorite',
  'foods',
  'on',
  'it',
  ',',
  'french',
  'fries',
  '.'],
 ['it', 'was', 'disgusting', 'food', ',', 'not', 'just', 'bad', 'food', '.'],
 ['a', 'plate', 'of', 'disgusting', 'food', 'found', 'at', 'a', 'diner', '.'],
 ['the',
  'meat',
  'on',
  'the',
  'burger',
  'looks',
  'like',
  'disgusting',
  'food',
  '.'],
 ['a',
  'plate',
  'of',
  'delicious',
  'food',
  'including',
  'french',
  'fries',
  '.'],
 ['french',
  'fries',
  'are',
  'not',
  'a',
  'healthy',
  'food',
  'but',
  'it',
  'is',
  'an',
  'excellent',
  'food',
  'for',
  'teenagers',
  '.'],
 ['the',
  'plate',
  'has',
  'one',
  'of',
  'my',
  'favorite',
  'foods',
  'on',
  'i

In [6]:
stop_words = set(stopwords.words('english'))
cleaned_raw = [[word for word in sentence if word not in stop_words] for sentence in raw_list]
cleaned_raw

[['plate', 'delicious', 'food', 'including', 'french', 'fries', '.'],
 ['french', 'fries', 'healthy', 'food', 'excellent', 'food', 'teenagers', '.'],
 ['plate', 'one', 'favorite', 'foods', ',', 'french', 'fries', '.'],
 ['disgusting', 'food', ',', 'bad', 'food', '.'],
 ['plate', 'disgusting', 'food', 'found', 'diner', '.'],
 ['meat', 'burger', 'looks', 'like', 'disgusting', 'food', '.'],
 ['plate', 'delicious', 'food', 'including', 'french', 'fries', '.'],
 ['french', 'fries', 'healthy', 'food', 'excellent', 'food', 'teenagers', '.'],
 ['plate', 'one', 'favorite', 'foods', ',', 'french', 'fries', '.'],
 ['disgusting', 'food', ',', 'bad', 'food', '.'],
 ['plate', 'disgusting', 'food', 'found', 'diner', '.'],
 ['meat', 'burger', 'looks', 'like', 'disgusting', 'food', '.'],
 ['plate', 'delicious', 'food', 'including', 'french', 'fries', '.'],
 ['french', 'fries', 'healthy', 'food', 'excellent', 'food', 'teenagers', '.'],
 ['plate', 'one', 'favorite', 'foods', ',', 'french', 'fries', '.'],

In [7]:
cleaned_raw_strings = [' '.join(sentence) for sentence in cleaned_raw]

Cvec = CountVectorizer(max_features=3)
Cvec.fit(cleaned_raw_strings)

print("Vocabulary:", Cvec.vocabulary_)

vector = Cvec.transform(cleaned_raw_strings)

print("Encoded Document is:")
print(vector.toarray())

Vocabulary: {'nice': 1, 'man': 0, 'people': 2}
Encoded Document is:
[[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [9]:
import torch
cleaned_raw_strings = [' '.join(sentence) for sentence in cleaned_raw]

Cvec = CountVectorizer(max_features=3)
Cvec.fit(cleaned_raw_strings)

print("Vocabulary:", Cvec.vocabulary_)

vector = Cvec.transform(cleaned_raw_strings)
text_tensor_list = torch.tensor(vector.toarray())

print("Encoded Document is:")
print(text_tensor_list)


Vocabulary: {'nice': 1, 'man': 0, 'people': 2}
Encoded Document is:
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [10]:
print(type(cleaned_raw_strings))

<class 'list'>


### Image Processing 

In [11]:
import os
from PIL import Image

folder_path = r"C:\Users\YOGA\Desktop\sentiment_images"
image_files = os.listdir(folder_path)

images = []
for filename in image_files:
    image_path = os.path.join(folder_path, filename)
    image = Image.open(image_path)
    images.append(image)


In [15]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os

# Define the folder path and initialize ResNet
folder_path = r"C:\Users\YOGA\Desktop\sentiment_images"
resnet = models.resnet50(pretrained=True)

# Set ResNet to evaluation mode
resnet.eval()

# Define the preprocessing transformation
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create a list to store the extracted features
features = []

# Extract visual features from each image
for image in images:
    # Convert image to RGB
    image = image.convert("RGB")
    
    # Apply the preprocessing transformation
    image = preprocess(image)
    
    # Expand the image dimensions to match ResNet's input shape
    image = image.unsqueeze(0)
    
    # Pass the image through ResNet and extract features
    with torch.no_grad():
        feature_vector = resnet(image)
    
    # Append the features to the list
    features.append(feature_vector)

# Concatenate the extracted features into a single tensor
features_tensor = torch.cat(features)

# Print the shape of the features tensor
print("Shape of features tensor:", features_tensor.shape)


Shape of features tensor: torch.Size([2221, 1000])


In [16]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import pandas as pd
import numpy as np

# Define the folder path and initialize ResNet
folder_path = r"C:\Users\YOGA\Desktop\sentiment_images"
resnet = models.resnet50(pretrained=True)

# Set ResNet to evaluation mode
resnet.eval()

# Define the preprocessing transformation
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataframe
df = pd.read_csv(r"C:\Users\YOGA\Desktop\sentiment.csv")

# Add a new column 'image_features' with default values of None
df['image_features'] = None


# Extract visual features from each image
for index, filename in enumerate(df['filename']):
    # Construct the image path
    image_path = os.path.join(folder_path, filename)
    
    # Check if the image file exists
    if os.path.exists(image_path):
        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image)
        image = image.unsqueeze(0)
        
        # Pass the image through ResNet and extract features
        with torch.no_grad():
            feature_vector = resnet(image)
        
        # Update the 'image_features' column with the extracted feature vector
        df.at[index, 'image_features'] = feature_vector.squeeze().numpy()

# Print the shape of the features tensor
print("Shape of features tensor:", df['image_features'].shape)

# Display the modified dataframe
print(df)


Shape of features tensor: (39199,)
       Unnamed: 0  imgid  split                       filename  successful  \
0               0  31369  train  COCO_val2014_000000389081.jpg           1   
1               1  31369  train  COCO_val2014_000000389081.jpg           1   
2               2  31369  train  COCO_val2014_000000389081.jpg           1   
3               3  31369  train  COCO_val2014_000000389081.jpg           1   
4               4  31369  train  COCO_val2014_000000389081.jpg           1   
...           ...    ...    ...                            ...         ...   
39194       39194  24628   test  COCO_val2014_000000190705.jpg           1   
39195       39195  24628   test  COCO_val2014_000000190705.jpg           1   
39196       39196  24628   test  COCO_val2014_000000190705.jpg           1   
39197       39197  24628   test  COCO_val2014_000000190705.jpg           1   
39198       39198  24628   test  COCO_val2014_000000190705.jpg           1   

                            

In [17]:
df

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw,image_features
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.,"[2.8987663, -1.0492762, -0.6302813, -0.5620507..."
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...,"[2.8987663, -1.0492762, -0.6302813, -0.5620507..."
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ...","[2.8987663, -1.0492762, -0.6302813, -0.5620507..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food.","[2.8987663, -1.0492762, -0.6302813, -0.5620507..."
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.,"[2.8987663, -1.0492762, -0.6302813, -0.5620507..."
...,...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...,"[-4.6079082, -1.1405089, -1.4482354, -2.753007..."
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.,"[-4.6079082, -1.1405089, -1.4482354, -2.753007..."
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom,"[-4.6079082, -1.1405089, -1.4482354, -2.753007..."
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...,"[-4.6079082, -1.1405089, -1.4482354, -2.753007..."


In [18]:
a = np.array([0.00])
print(type(a))

<class 'numpy.ndarray'>


In [19]:
a = np.array([0.00])

# Assuming "df" is your pandas DataFrame
df['image_features'].fillna(value=a[0], inplace=True)
print(df['image_features'])

0        [2.8987663, -1.0492762, -0.6302813, -0.5620507...
1        [2.8987663, -1.0492762, -0.6302813, -0.5620507...
2        [2.8987663, -1.0492762, -0.6302813, -0.5620507...
3        [2.8987663, -1.0492762, -0.6302813, -0.5620507...
4        [2.8987663, -1.0492762, -0.6302813, -0.5620507...
                               ...                        
39194    [-4.6079082, -1.1405089, -1.4482354, -2.753007...
39195    [-4.6079082, -1.1405089, -1.4482354, -2.753007...
39196    [-4.6079082, -1.1405089, -1.4482354, -2.753007...
39197    [-4.6079082, -1.1405089, -1.4482354, -2.753007...
39198    [-4.6079082, -1.1405089, -1.4482354, -2.753007...
Name: image_features, Length: 39199, dtype: object


In [21]:
import torch
import pandas as pd
# Assuming "df" is your pandas DataFrame
df['image_features'].fillna(value=a[0], inplace=True)

# Convert the "image_features" column to a tensor list
image_tensor_list = []
for value in df['image_features']:
    tensor_value = torch.tensor(value)
    image_tensor_list.append(tensor_value)

# Print the tensor list
print(image_tensor_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
print(len(image_tensor_list))

39199


In [23]:
sentiment_column = df['sentiment']
sentiment_list = sentiment_column.tolist()
sentiment_tensor_list = torch.tensor(sentiment_list)
print(sentiment_tensor_list)

tensor([1, 1, 1,  ..., 0, 0, 0])


In [24]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Create a list to store the padded tensors
padded_image_tensors = []
padded_text_tensors = []

# Iterate over image_tensor_list and text_tensor_list
for image_tensor in image_tensor_list:
    if image_tensor.dim() == 0:
        # Add a dummy dimension to tensors with no dimensions
        image_tensor = image_tensor.unsqueeze(0)
    padded_image_tensors.append(image_tensor)

for text_tensor in text_tensor_list:
    if text_tensor.dim() == 0:
        # Add a dummy dimension to tensors with no dimensions
        text_tensor = text_tensor.unsqueeze(0)
    padded_text_tensors.append(text_tensor)

# Convert padded_image_tensors and padded_text_tensors to padded sequences
padded_images = pad_sequence(padded_image_tensors, batch_first=True)
padded_texts = pad_sequence(padded_text_tensors, batch_first=True)

# Print the padded sequences
print("Padded Images:\n", padded_images)
print("\nPadded Texts:\n", padded_texts)


Padded Images:
 tensor([[ 2.8988, -1.0493, -0.6303,  ...,  4.0891,  3.7832, -0.8168],
        [ 2.8988, -1.0493, -0.6303,  ...,  4.0891,  3.7832, -0.8168],
        [ 2.8988, -1.0493, -0.6303,  ...,  4.0891,  3.7832, -0.8168],
        ...,
        [-4.6079, -1.1405, -1.4482,  ..., -1.4821,  0.4565, 11.9301],
        [-4.6079, -1.1405, -1.4482,  ..., -1.4821,  0.4565, 11.9301],
        [-4.6079, -1.1405, -1.4482,  ..., -1.4821,  0.4565, 11.9301]])

Padded Texts:
 tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [25]:
from sklearn.model_selection import train_test_split
# split the data into training and testing before being used
images_train, images_test, texts_train, texts_test, sentiments_train, sentiments_test = train_test_split(padded_images, padded_texts, sentiment_tensor_list, test_size=0.2, random_state=50)

In [61]:
#complete model with scores
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Define the multimodal model
class MyMultimodalModel(nn.Module):
    def __init__(self, text_input_size, image_input_size, hidden_size):
        super(MyMultimodalModel, self).__init__()

        # Define the hidden layers for text input
        self.text_hidden1 = nn.Linear(text_input_size, hidden_size)
        self.text_hidden2 = nn.Linear(hidden_size, hidden_size)
        self.dropout_text = nn.Dropout(0.3)  # Add dropout layer for text

        # Define the hidden layers for image input
        self.image_hidden1 = nn.Linear(image_input_size, hidden_size)
        self.image_hidden2 = nn.Linear(hidden_size, hidden_size)
        self.dropout_image = nn.Dropout(0.3)  # Add dropout layer for image

        # Define the subsequent hidden layers
        self.hidden3 = nn.Linear(hidden_size * 2, hidden_size)
        self.hidden4 = nn.Linear(hidden_size, hidden_size)
        self.hidden5 = nn.Linear(hidden_size, hidden_size)

        # Define the output layer
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, text_input, image_input):
        # Process text input through the hidden layers
        text_output = self.dropout_text(nn.ReLU()(self.text_hidden1(text_input)))
        text_output = nn.ReLU()(self.text_hidden2(text_output))

        # Process image input through the hidden layers
        image_output = self.dropout_image(nn.ReLU()(self.image_hidden1(image_input)))
        image_output = nn.ReLU()(self.image_hidden2(image_output))

        # Merge the text and image features
        merged_features = torch.cat((text_output, image_output), dim=1)

        # Process through the subsequent hidden layers
        output = nn.ReLU()(self.hidden3(merged_features))
        output = nn.ReLU()(self.hidden4(output))
        output = nn.ReLU()(self.hidden5(output))

        # Pass through the output layer
        output = self.output(output)

        return output

# Split the data into training and testing sets
images_train, images_test, texts_train, texts_test, sentiments_train, sentiments_test = train_test_split(
    padded_images, padded_texts, sentiment_tensor_list, test_size=0.2, random_state=50)

# Convert the training data into tensors
images_train = torch.tensor(images_train, dtype=torch.float32)
texts_train = torch.tensor(texts_train, dtype=torch.float32)
sentiments_train = torch.tensor(sentiments_train, dtype=torch.float32).unsqueeze(1)

# Get the input sizes
text_input_size = texts_train.shape[1]
image_input_size = images_train.shape[1]
hidden_size = 128  # Increase the hidden size

# Instantiate the multimodal model
model = MyMultimodalModel(text_input_size, image_input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # Use Adam optimizer and L2 regularization


# Convert the test data into tensors
images_test = torch.tensor(images_test, dtype=torch.float32)
texts_test = torch.tensor(texts_test, dtype=torch.float32)
sentiments_test = torch.tensor(sentiments_test, dtype=torch.float32).unsqueeze(1)

# Train the model
best_loss = float('inf')
patience = 20
for epoch in range(250):
    # Forward pass
    outputs = model(texts_train, images_train)
    loss = criterion(outputs, sentiments_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate train accuracy
    train_predictions = torch.round(outputs).squeeze()
    train_accuracy = (train_predictions == sentiments_train.squeeze()).sum().item() / len(sentiments_train) * 100

    # Calculate test accuracy and F1 score
    with torch.no_grad():
        model.eval()
        test_outputs = model(texts_test.to(texts_train.device), images_test.to(images_train.device))
        test_predictions = torch.round(test_outputs).squeeze()
        test_accuracy = (test_predictions == sentiments_test.squeeze()).sum().item() / len(sentiments_test) * 100
        f1 = f1_score(sentiments_test.squeeze().cpu().numpy(), test_predictions.cpu().numpy(), average='weighted') * 100

        model.train()

    # Print progress
    if (epoch + 1) % 10 == 0:
        
        print("Epoch [{}/{}], Loss = {:.4f}, Train Accuracy = {:.4f}%, Test Accuracy = {:.4f}%, F1 Score = {:.4f}".format(
            epoch + 1, 250, loss.item(), train_accuracy, test_accuracy, f1))

C:\Users\YOGA\AppData\Local\Temp\ipykernel_8812\2114899504.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images_train = torch.tensor(images_train, dtype=torch.float32)
C:\Users\YOGA\AppData\Local\Temp\ipykernel_8812\2114899504.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  texts_train = torch.tensor(texts_train, dtype=torch.float32)
C:\Users\YOGA\AppData\Local\Temp\ipykernel_8812\2114899504.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentiments_train = torch.tensor(sentiments_train, dtype=torch.float32).unsqueeze(1)


Epoch [10/250], Loss = 0.2684, Train Accuracy = 47.3197%, Test Accuracy = 47.2066%, F1 Score = 36.1973
Epoch [20/250], Loss = 0.2470, Train Accuracy = 54.2109%, Test Accuracy = 52.7168%, F1 Score = 48.6736
Epoch [30/250], Loss = 0.2352, Train Accuracy = 56.8704%, Test Accuracy = 56.6582%, F1 Score = 55.2188
Epoch [40/250], Loss = 0.2288, Train Accuracy = 56.9087%, Test Accuracy = 56.1097%, F1 Score = 52.3906
Epoch [50/250], Loss = 0.2270, Train Accuracy = 57.5879%, Test Accuracy = 56.7602%, F1 Score = 54.8038
Epoch [60/250], Loss = 0.2259, Train Accuracy = 58.5765%, Test Accuracy = 57.5893%, F1 Score = 55.5626
Epoch [70/250], Loss = 0.2236, Train Accuracy = 59.8361%, Test Accuracy = 58.0357%, F1 Score = 55.4361
Epoch [80/250], Loss = 0.2199, Train Accuracy = 60.9937%, Test Accuracy = 58.8138%, F1 Score = 56.4772
Epoch [90/250], Loss = 0.2139, Train Accuracy = 63.1876%, Test Accuracy = 60.2806%, F1 Score = 60.0350
Epoch [100/250], Loss = 0.2102, Train Accuracy = 64.3005%, Test Accuracy 